<a href="https://colab.research.google.com/github/ced-sys/.py/blob/main/Crop_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
base_path='/content/drive/MyDrive/cropland-mapping'
folders=['data', 'notebooks', 'submissions', 'src']

In [ ]:
print("Setting up project structure...")
for folder in folders:
  folder_path=os.path.join(base_path, folder)
  os.makedirs(folder_path, exist_ok=True)
  print(f"Created: {folder_path}")

In [ ]:
zindi_path='/content/drive/MyDrive/Zindi Hackathons'
if os.path.exists(zindi_path):
  print(f"Found zindi hackathons folder at : {zindi_path}")
  print("Contents:")
  for item in os.listdir(zindi_path):
    print(f" -{item}")
else:
  print("Zindi Hackathons folder not found. Please check the path")